<a href="https://colab.research.google.com/github/adsc3/covid19-tokyo-extrapolation/blob/master/covid19_tokyo_extrapolate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>